# Thumbs - Live Demo

In this notebook we'll use the model we trained to detect whether the thumb is ``up`` or ``down``.

## Load the trained model

We'll assumed that you've already trained the model a save it in a file called ``best_model.pth``.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device, if enabled.

In [ ]:
device_string = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_string)
model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that thumbs is up or down

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
thumbs_slider = widgets.FloatSlider(description='thumbs up', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

file = open("pictures/thumbs-updown-sign_224.png", "rb")
image_thumbs = file.read()

display(widgets.HBox([image, thumbs_slider, widgets.Image(value=image_thumbs, format='png', width=160, height=224,)]))

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global thumbs_slider
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_thumbsdown = float(y.flatten()[0])
    thumbs_slider.value = (1 - prob_thumbsdown)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Cool! We've created our neural network execution function, and attaching it to the camera for processing.
We accomplish that with the ``observe`` function.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
camera.unobserve(update, names='value')

### Conclusion

That's it for this live demo!  Hopefully you had some fun.